##teams

In [2]:

import os
import sys
import plotly.graph_objects as go

from tqdm.notebook import tqdm

import math

import pandas as pd
import json
pd.set_option('display.max_columns', None)

from io import BytesIO
from pathlib import Path

import socceraction.spadl as spadl
import socceraction.spadl.wyscout as wyscout
import json

In [3]:
with open('files/teams.json', 'r') as f:
    data = json.load(f)
df_teams = pd.DataFrame(data)

with open('files/events_England.json', 'r') as f:
    data = json.load(f)
df_events = pd.DataFrame(data)

In [23]:
# To-dos:
    # Fazer "de-para" (busca em campos json) com as colunas colunas: playerId, matchID, teamID
    # Ao buscar o matchID dá pra criar uma coluna data do jogo tb
    # remover colunas eventId, subEventName, id

In [4]:
df_events

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175
...,...,...,...,...,...,...,...,...,...,...,...,...
643145,5,Ball out of the field,[],0,"[{'y': 32, 'x': 0}, {'y': 100, 'x': 100}]",2500098,Interruption,1623,2H,2796.732525,50,251596409
643146,3,Corner,"[{'id': 302}, {'id': 801}, {'id': 1801}]",70965,"[{'y': 100, 'x': 100}, {'y': 47, 'x': 88}]",2500098,Free Kick,1633,2H,2829.821084,30,251596232
643147,1,Air duel,"[{'id': 701}, {'id': 1802}]",7919,"[{'y': 53, 'x': 12}, {'y': 50, 'x': 14}]",2500098,Duel,1623,2H,2831.211419,10,251596410
643148,1,Air duel,"[{'id': 703}, {'id': 1801}]",8005,"[{'y': 47, 'x': 88}, {'y': 50, 'x': 86}]",2500098,Duel,1633,2H,2832.434399,10,251596234


In [5]:
df_teams.head()

,city,name,wyId,officialName,area,type
0,Newcastle upon Tyne,Newcastle United,1613,Newcastle United FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
1,Vigo,Celta de Vigo,692,Real Club Celta de Vigo,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
2,Barcelona,Espanyol,691,Reial Club Deportiu Espanyol,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
3,Vitoria-Gasteiz,Deportivo Alav\u00e9s,696,Deportivo Alav\u00e9s,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club
4,Valencia,Levante,695,Levante UD,"{'name': 'Spain', 'id': '724', 'alpha3code': '...",club


In [6]:
df = pd.merge(df_events, df_teams, left_on='teamId', right_on='wyId', how='left')

In [8]:
df.head()

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,city,name,wyId,officialName,area,type
0,8,Simple pass,[{'id': 1801}],25413,"[{'y': 49, 'x': 49}, {'y': 78, 'x': 31}]",2499719,Pass,1609,1H,2.758649,85,177959171,London,Arsenal,1609,Arsenal FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
1,8,High pass,[{'id': 1801}],370224,"[{'y': 78, 'x': 31}, {'y': 75, 'x': 51}]",2499719,Pass,1609,1H,4.946850,83,177959172,London,Arsenal,1609,Arsenal FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
2,8,Head pass,[{'id': 1801}],3319,"[{'y': 75, 'x': 51}, {'y': 71, 'x': 35}]",2499719,Pass,1609,1H,6.542188,82,177959173,London,Arsenal,1609,Arsenal FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
3,8,Head pass,[{'id': 1801}],120339,"[{'y': 71, 'x': 35}, {'y': 95, 'x': 41}]",2499719,Pass,1609,1H,8.143395,82,177959174,London,Arsenal,1609,Arsenal FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club
4,8,Simple pass,[{'id': 1801}],167145,"[{'y': 95, 'x': 41}, {'y': 88, 'x': 72}]",2499719,Pass,1609,1H,10.302366,85,177959175,London,Arsenal,1609,Arsenal FC,"{'name': 'England', 'id': '0', 'alpha3code': '...",club


In [9]:
df['subEventName'].unique()

array(['Simple pass', 'High pass', 'Head pass', 'Air duel',
       'Ground loose ball duel', 'Smart pass', 'Launch',
       'Ground defending duel', 'Ground attacking duel', 'Foul',
       'Free Kick', 'Cross', 'Shot', 'Reflexes', 'Touch', 'Clearance',
       'Ball out of the field', 'Throw in', 'Goal kick', 'Corner',
       'Goalkeeper leaving line', 'Hand pass', 'Acceleration',
       'Save attempt', '', 'Free kick cross', 'Free kick shot',
       'Hand foul', 'Violent Foul', 'Protest', 'Whistle',
       'Late card foul', 'Out of game foul', 'Penalty', 'Time lost foul',
       'Simulation'], dtype=object)

In [10]:
#countig number o pass by teams
df_filtered = df[df['subEventName'].isin(['Head pass', 'Simple pass', 'High pass', 'Smart pass', 'Hand pass'])]
df_total_eventos = pd.pivot_table(df_filtered, index='name', columns='subEventName', aggfunc='size', fill_value=0)
df_total_eventos

subEventName,Hand pass,Head pass,High pass,Simple pass,Smart pass
name,,,,,
AFC Bournemouth,188,1105,1098,11653,304
Arsenal,221,1001,1187,18995,505
Brighton & Hove Albion,109,1087,1265,10172,198
Burnley,111,1322,1309,8484,159
Chelsea,169,854,1243,16751,377
Crystal Palace,89,943,1086,9989,252
Everton,135,1117,1314,10142,187
Huddersfield Town,80,1104,1300,10249,166
Leicester City,107,1275,1403,10258,242


In [24]:
df_total_eventos['total'] = df_total_eventos['Hand pass'] + df_total_eventos['Head pass'] + df_total_eventos['High pass'] + df_total_eventos['Simple pass'] + df_total_eventos['Smart pass'] 

In [39]:
df_total_eventos.columns

Index(['Hand pass', 'Head pass', 'High pass', 'Simple pass', 'Smart pass',
       'total', 'mean'],
      dtype='object', name='subEventName')

In [27]:
df_total_eventos['mean'] = df_total_eventos['total']/38
df_total_eventos


subEventName,Hand pass,Head pass,High pass,Simple pass,Smart pass,total,mean
name,,,,,,,
AFC Bournemouth,188,1105,1098,11653,304,14348,377.578947
Arsenal,221,1001,1187,18995,505,21909,576.552632
Brighton & Hove Albion,109,1087,1265,10172,198,12831,337.657895
Burnley,111,1322,1309,8484,159,11385,299.605263
Chelsea,169,854,1243,16751,377,19394,510.368421
Crystal Palace,89,943,1086,9989,252,12359,325.236842
Everton,135,1117,1314,10142,187,12895,339.342105
Huddersfield Town,80,1104,1300,10249,166,12899,339.447368
Leicester City,107,1275,1403,10258,242,13285,349.605263


In [38]:
# Criação do gráfico de barras
fig = go.Figure(data=go.Bar(x=df_total_eventos.index, y=df_total_eventos['mean']))

# Configurações do layout
fig.update_layout(title='Média de Passes por Time',
                  xaxis_title='Time',
                  yaxis_title='Média de Passes')

# Exibição do gráfico
fig.show()